## **Standard U-Net Channel Flow (base=64):**

### **Encoder Flow:**

| Level | Input | Conv1 Output | Conv2 Output | Pool Output | Spatial |
|-------|-------|--------------|--------------|-------------|---------|
| **Input** | 3 | - | - | - | 256×256 |
| **E1** | 3 | → **64** | → **64** | → **64** | 256×256 → 128×128 |
| **E2** | 64 | → **128** | → **128** | → **128** | 128×128 → 64×64 |
| **E3** | 128 | → **256** | → **256** | → **256** | 64×64 → 32×32 |
| **E4** | 256 | → **512** | → **512** | → **512** | 32×32 → 16×16 |
| **Bottleneck** | 512 | → **1024** | → **1024** | - | 16×16 |

### **Decoder Flow:**

| Level | Input | Upsample | Concat Skip | Conv1 Output | Conv2 Output | Spatial |
|-------|-------|----------|-------------|--------------|--------------|---------|
| **D4** | 1024 | → 512 | +512 = 1024 | → **512** | → **512** | 32×32 |
| **D3** | 512 | → 256 | +256 = 512 | → **256** | → **256** | 64×64 |
| **D2** | 256 | → 128 | +128 = 256 | → **128** | → **128** | 128×128 |
| **D1** | 128 | → 64 | +64 = 128 | → **64** | → **64** | 256×256 |
| **Final** | 64 | - | - | → **num_classes** | - | 256×256 |

## **Complete Architecture Visualization:**

```
Input: 256×256×3
  ↓
[E1] 256×256×64 ──┐
  ↓ pool          │ Skip Connection
128×128×64        │
  ↓               │
[E2] 128×128×128 ─┼──┐
  ↓ pool          │  │ Skip Connection  
64×64×128         │  │
  ↓               │  │
[E3] 64×64×256  ──┼──┼──┐
  ↓ pool          │  │  │ Skip Connection
32×32×256         │  │  │
  ↓               │  │  │
[E4] 32×32×512  ──┼──┼──┼──┐
  ↓ pool          │  │  │  │ Skip Connection
16×16×512         │  │  │  │
  ↓               │  │  │  │
[Bottleneck] 16×16×1024
  ↓
[D4] 32×32×512  ←──┼──┼──┼── (Upsample + Concat)
  ↓                │  │  │
[D3] 64×64×256  ←──┼──┼── (Upsample + Concat)
  ↓                │  │
[D2] 128×128×128 ←─┼── (Upsample + Concat)
  ↓                │
[D1] 256×256×64  ←── (Upsample + Concat)
  ↓
[Final] 256×256×num_classes
```


## U-Net Encoder-decoder for medical image segmentation

## nnU-Net Self-adapting framework for medical images